머신러닝에서는 에러율로 판별


평균값에서 실제값까지의 차이 절대값 > 편차

그래서 얘를 제곱해버려! > 분산
루트> 표준편차


실제데이터와 추정치간의 차이로 대입해서 생각해보자

mae > 잔차들의 절대값 평균 >> 이상치에 덜 민감
mse > 제곱 (정사각형 ) >> 이상치에 민감
rmse > 

print("평균절대오차(MAE): ", mean_absolute_error(y_test, y_pred))



mpe : 절대값이 없어서 -,+ 부호가 있어서 경향성 확인 가능

95%로 장담할 수 있따, 100문제 중에 95문제를 맞추는게아니라 ,1문제를 맞췄을 떄, 95% 장담할 수 있다


오답률 얼마나 많이 틀리는가...
보통 rmse 로 봄


이모형이 과연진실만 말하는가?


학습 곡선으로 과적합 판정


# [LAB 06] 지도학습 > 예측 > 선형 > 03-성능 평가 지표
## 농어의 무게 예측 모형
## #01. 준비 작업
### [1] 패키지 참조

In [1]:
from hossam import *
from pandas import DataFrame, read_excel
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
import statsmodels as sm

from itertools import combinations

#데이터 표준화 모듈
from sklearn.preprocessing import StandardScaler

#선형 회귀 분석 모듈
from sklearn.linear_model import LinearRegression

#훈련/검증 데이터 분리 모듈
from sklearn.model_selection import train_test_split


from scipy.stats import pearsonr,spearmanr


#성능 평가 지표 모듈
from sklearn.metrics import(
  r2_score,
  mean_absolute_error,
  mean_squared_error,
  mean_absolute_percentage_error
)

### [2] 데이터 가져오기
- 이전 실습에서 전처리가 완료된 데이터셋에서 이어서 진행합니다

In [2]:
origin = load_data('fish_processed')
origin.head()

농어의 길이,높이,두께,무게를 조사한 데이터의 전처리 버전


,길이,높이,두께,무게
0,-2.180225,-2.016507,-1.896175,1.931521
1,-1.587434,-1.518703,-1.560774,3.496508
2,-1.442032,-1.417039,-1.316328,3.713572
3,-1.307815,-1.147103,-1.202633,3.960813
4,-1.173599,-1.147103,-1.026405,4.262680


### [3] 학습/검증 데이터 분할

In [3]:
df = origin

yname='무게'
x=df.drop(columns=[yname])
y=df[yname]

x_train,x_test,y_train,y_test = train_test_split(

  x,y,test_size=0.25,random_state=52
)

x_train.shape,x_test.shape,y_train.shape,y_test.shape

((42, 3), (14, 3), (42,), (14,))

### [4] 선형회귀 모형 적합

In [4]:
estimator = LinearRegression(n_jobs=-1)
estimator.fit(x_train,y_train)
estimator

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",-1
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


### [5] 검증 데이터의 추정치 얻기

In [5]:
y_pred=estimator.predict(x_test)
y_pred[:5]


array([5.54122557, 4.09686775, 4.20993642, 5.01685527, 6.84072283])

## #02. 회귀분석 모델의 성능 평가
- 회귀분석 모델의 평가를 위한 지표는 검증데이터의 관측치와 그에 대한 추정치의 차이를 기반으로 한다
  
| 구분  | 지표   | 영어 이름                          | 한글 이름        | 목적                       | 사용하는 경우                                     |
| --- | ---- | ------------------------------ | ------------ | ------------------------ | ------------------------------------------- |
| 설명력 | R²   | Coefficient of Determination   | 결정계수         | 종속변수 분산 중 모델이 설명하는 비율 측정 | • 설명력 중심 분석<br>• 모델 간 설명력 비교                |
| 에러율 | MAE  | Mean Absolute Error            | 평균 절대 오차     | 예측 오차의 평균 크기 측정          | • 오차 크기를 직관적으로 보고 싶을 때<br>• 이상치 영향 최소화 필요 시 |
| 에러율 | MSE  | Mean Squared Error             | 평균 제곱 오차     | 오차 제곱의 평균 측정             | • 수학적 최적화 목적<br>• 모델 학습 목적함수 설명 시           |
| 에러율 | RMSE | Root Mean Squared Error        | 평균 제곱근 오차    | 큰 오차를 반영한 평균 오차 크기 측정    | • 큰 오차를 중요하게 볼 때<br>• MSE와 동일 단위를 써야 할 때    |
| 에러율 | MAPE | Mean Absolute Percentage Error | 평균 절대 백분율 오차 | 실제값 대비 오차의 비율(%) 측정      | • 스케일이 다른 데이터 비교<br>• 상대적 오차 크기 표현          |
| 에러율 | MPE  | Mean Percentage Error          | 평균 비율 오차     | 오차의 평균 방향(+/−) 측정        | • 과대/과소 예측 경향 확인<br>• 편향 방향 점검              |


### [1] 설명력 R2 결정계수
- 회귀분석에서 가장 많이 채택되는 설명력 값

#### (1) model.score() 메서드 사용
- 기본적으로 모델의 학습 결과를 갖고 있는 fit 객체의 score() 메서드에 독립변수와 종속변수를 전달하여 확인할 수 있다

In [6]:
print('설명력(R2)',estimator.score(x_test,y_test))

설명력(R2) 0.9521204045190643


#### (2) r2_score() 메서드 사용
- sklearn 이 제공하는 metrics 객체의 r2_score() 메서드에 검증 데이터의 관측치와 추정치를 전달하여 조회할 수 있다

In [7]:
print('설명력(R2)',r2_score(y_test,y_pred))

설명력(R2) 0.9521204045190643


### [2] 에러율
#### (1) MAE (Mean Absolute Error) : 평균 절대 오차

모델의 추정치와 관측치의 차이에 대한 절대값의 평균이다.
절대값을 취하기 때문에 직관적으로 이해하기 쉬운 지표다.
이상치의 영향을 크게 받지 않는다.
MAE는 절대값을 사용하는 지표이기 때문에 실제보다 낮게 예측했는지(underperformance) 높게 예측했는지(overperformance) 는 알 수 없다.

In [8]:
print("평균절대오차(MAE): ", mean_absolute_error(y_test, y_pred))

평균절대오차(MAE):  0.17570080101045002


#### (2) MSE (Mean Squared Error) : 평균 제곱 오차
MAE 와는 다르게 제곱을 하기 때문에 모델의 실제값과 예측값의 차이의 면적의 합
제곱을 하기 때문에 값이 커질 수 있음
이상치가 존재하면 수치가 많이 늘어남 (이상치에 민감함)

In [ ]:
print("평균제곱오차(MSE): ", mean_squared_error(y_test, y_pred))

평균절대오차(MAE):  0.05557567448278762


#### (3) RMSE (Root Mean Squared Error) : 평균 오차
RMSE = MSE 를 구한 값이 루트를 씌운 값
오류 지표를 실제 값과 유사한 단위로 변환하여 해석을 쉽게할 수 있음

In [11]:
print("평균오차(RMSE): ", np.sqrt(mean_squared_error(y_test, y_pred)))

평균오차(RMSE):  0.23574493522192078


#### (4) MAPE (Mean Absolute Percentage Error) : 평균 절대 백분오차 비율
MAPE = MAE 를 퍼센트로 변환한 것
MAE 와 동일하게 MSE보다 이상치에 민감하며, 실제값보다 낮은 값인지 높은 값인지 알 수 없다
모델에 대한 편향이 있따 (이를 대응하기 위해 MPE도 추가로 확인하는 것을 추천함)

In [12]:
print("평균 절대 백분오차 비율 (MAPE): ", mean_absolute_percentage_error(y_test, y_pred))

평균 절대 백분오차 비율 (MAPE):  0.030129159825653062


#### (5) MPE(Mean Percentage Error) : 평균 비율 오차

MPE =
MAPE 와 비슷하지만 MAPE 에서 절대값을 제외한 지표
장점은 모델이 실제값보다 낮은 값인지 큰 값인지 판단할 수 있다

In [13]:
print("평균 비율 오차(MAE): ", np.mean(y_test-y_pred / y_test*100))

평균 비율 오차(MAE):  -93.6935422175235


### [3] 성능평가지표 일괄 확인

In [14]:
result = {
    "결정계수(R2)": r2_score(y_test, y_pred),
    "평균절대오차(MAE)": mean_absolute_error(y_test, y_pred),
    "평균제곱오차(MSE)": mean_squared_error(y_test, y_pred),
    "평균제곱근오차(RMSE)": np.sqrt(mean_squared_error(y_test, y_pred)),
    "평균 절대 백분오차(MAPE)": mean_absolute_percentage_error(y_test, y_pred),
    "평균 비율 오차(MPE)": np.mean((y_test - y_pred) / y_test * 100),
}

result_df = DataFrame(result, index=["성능지표"])
result_df


,결정계수(R2),평균절대오차(MAE),평균제곱오차(MSE),평균제곱근오차(RMSE),평균 절대 백분오차(MAPE),평균 비율 오차(MPE)
성능지표,0.95212,0.175701,0.055576,0.235745,0.030129,0.7438
